In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import collections
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,r2_score, mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split  
from imblearn.over_sampling import SMOTE

from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesRegressor

In [2]:
f = open("/Users/shreyassrinivasan/Desktop/SF_Project_Alias/merge_final.csv")
df = pd.read_csv(f)

In [3]:
df = df.dropna()

In [4]:
x = df.drop(['Unnamed: 0','Unnamed: 0_x','DepDel15','ArrTime','ArrDelayMinutes','ArrDel15',
            'ArrFlightDate','ArrAbsTime','DepAbsTime','Unnamed: 0_y','date_x','time_x','airport_x','Unnamed: 0.1',
            'date_y','time_y','airport_y','FlightDate','OriginAirportID','DestAirportID','DepTime'],axis=1)

In [5]:
flightOrigin = {'DFW':1,'EWR':2,'IAH':3,'JFK':4,'LAS':5,'LAX':6,'MCO':7,'MIA':8,'ORD':9,'PHX':10,'SEA':11,'SFO':12,'ATL':13,'CLT':14,'DEN':15} 

In [6]:
arr1 = [df['Origin'].apply(lambda x: flightOrigin.get(x))]

In [7]:
arr2 = [df['Dest'].apply(lambda x: flightOrigin.get(x))]

In [8]:
x['newOrigin'] = arr1[0]

In [9]:
x['newDest'] = arr2[0]

In [10]:
x = x.drop(['Origin','Dest'],axis=1)

In [11]:
x.transpose()

,0,1,2,3,4,5,6,7,8,9,...,1877184,1877185,1877186,1877187,1877188,1877189,1877190,1877191,1877192,1877193
Year,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,...,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0
Quarter,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Month,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
DayofMonth,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,13.0,18.0,25.0,4.0,11.0,25.0,19.0,18.0,25.0,26.0
CRSDepTime,1400.0,1355.0,1100.0,1045.0,1235.0,1240.0,1420.0,1141.0,1510.0,1500.0,...,2250.0,2245.0,2245.0,2326.0,2328.0,2328.0,550.0,550.0,550.0,550.0
DepDelayMinutes,4.0,0.0,0.0,80.0,0.0,0.0,0.0,2.0,4.0,24.0,...,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CRSArrTime,1601.0,1617.0,1652.0,1647.0,1600.0,1606.0,1645.0,1610.0,1617.0,1629.0,...,200.0,158.0,158.0,55.0,58.0,58.0,655.0,655.0,655.0,655.0
windspeedKmph_x,15.0,14.0,5.0,5.0,7.0,7.0,17.0,5.0,15.0,23.0,...,28.0,16.0,25.0,15.0,20.0,6.0,3.0,18.0,3.0,5.0
winddirDegree_x,237.0,138.0,111.0,35.0,248.0,248.0,174.0,121.0,292.0,256.0,...,309.0,222.0,243.0,171.0,188.0,122.0,201.0,138.0,97.0,335.0
weatherCode_x,119.0,353.0,113.0,113.0,122.0,122.0,113.0,116.0,113.0,113.0,...,113.0,116.0,176.0,122.0,113.0,122.0,122.0,296.0,113.0,113.0


In [12]:
y = df['ArrDel15']

In [13]:
x.shape

(1851334, 33)

In [14]:
y.shape

(1851334,)

In [15]:
np.any(np.isnan(x))

False

In [16]:
np.all(np.isfinite(x))

True

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

In [19]:
smote = SMOTE()
x_resampled,y_resampled = smote.fit_resample(x_train,y_train)

In [21]:
# Classification

xgb = XGBClassifier()
xgb.fit(x_resampled,y_resampled)

/Users/shreyassrinivasan/miniconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:37:03] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
pred = xgb.predict(x_test)

In [23]:
cm = confusion_matrix(y_test,pred)
cm

array([[284011,   8294],
       [ 22380,  55582]])

In [24]:
# Accuracy

score = xgb.score(x_test,y_test)
print(score)

0.9171570785406207


In [25]:
# Classification Report

target_names=["Class 0","Class 1"]
print(classification_report(y_test,pred,target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.93      0.97      0.95    292305
     Class 1       0.87      0.71      0.78     77962

    accuracy                           0.92    370267
   macro avg       0.90      0.84      0.87    370267
weighted avg       0.91      0.92      0.91    370267



In [ ]:
# Regression

In [18]:
x['ArrDel15'] = df['ArrDel15']

In [19]:
X = x.loc[x['ArrDel15'] == 1]

In [20]:
X = X.drop(['ArrDel15'],axis=1)

In [21]:
X.transpose()

,3,9,13,18,19,22,28,29,30,32,...,1877138,1877140,1877152,1877163,1877174,1877178,1877179,1877181,1877182,1877184
Year,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,...,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0
Quarter,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Month,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
DayofMonth,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,6.0,5.0,3.0,18.0,12.0,11.0,3.0,6.0,10.0,13.0
CRSDepTime,1045.0,1500.0,1425.0,1410.0,1410.0,805.0,1400.0,1415.0,1055.0,1205.0,...,550.0,2150.0,550.0,2328.0,550.0,2140.0,2250.0,2250.0,2250.0,2250.0
DepDelayMinutes,80.0,24.0,45.0,68.0,24.0,45.0,44.0,49.0,49.0,44.0,...,108.0,30.0,280.0,0.0,6.0,22.0,18.0,33.0,18.0,30.0
CRSArrTime,1647.0,1629.0,1532.0,1648.0,1649.0,1630.0,1657.0,1630.0,1630.0,1624.0,...,755.0,7.0,755.0,58.0,755.0,100.0,200.0,200.0,200.0,200.0
windspeedKmph_x,5.0,23.0,15.0,15.0,15.0,5.0,15.0,16.0,8.0,15.0,...,20.0,16.0,22.0,12.0,3.0,8.0,20.0,10.0,14.0,28.0
winddirDegree_x,35.0,256.0,237.0,237.0,237.0,95.0,140.0,295.0,220.0,90.0,...,214.0,165.0,195.0,87.0,83.0,205.0,290.0,170.0,213.0,309.0
weatherCode_x,113.0,113.0,119.0,119.0,119.0,113.0,116.0,113.0,122.0,113.0,...,353.0,113.0,302.0,113.0,248.0,122.0,122.0,113.0,122.0,113.0


In [22]:
Y = df['ArrDelayMinutes']

In [23]:
arr = []
for i in X.index:
    arr.append(Y[i])
Y = pd.Series(arr)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

In [25]:
extTree = ExtraTreesRegressor(n_estimators=100, random_state = 42)

In [26]:
extTree.fit(x_train,y_train)

ExtraTreesRegressor(random_state=42)

In [27]:
pred_ext = extTree.predict(x_test)

In [28]:
print("R2: " , r2_score(y_test,pred_ext))
print("Root Mean Square Error: ", np.sqrt(mean_squared_error(y_test,pred_ext)))
print("Mean Absolute Error: ", mean_absolute_error(y_test,pred_ext))

R2:  0.9480903869440628
Root Mean Square Error:  16.54923671647846
Mean Absolute Error:  11.64494233841872
